In [1]:
%matplotlib tk 
import mne
import numpy as np
import matplotlib.pyplot as plt
from kymatio.sklearn import Scattering2D
from kymatio.scattering2d.utils import fft2
from colorsys import hls_to_rgb

In [2]:
epoch_file = '../preproc/Physionet_raw_epo.fif'
epochs = mne.read_epochs(epoch_file)
epochs = epochs.apply_baseline()
epochs.equalize_event_counts()

Reading ../preproc/Physionet_raw_epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Dropped 2 epochs: 72, 85


(<EpochsFIF |  88 events (all good), -2 - 4 sec, baseline -2 – 0 sec, ~17.5 MB, data loaded,
  'T1': 44
  'T2': 44>,
 array([72, 85]))

In [43]:
tmin, tmax = -0.5, 2
epochs_data = epochs.get_data()
# labels = epochs.events[epochs.events[:,-1]!=3]
labels = epochs.events[:,-1]
labels = labels-2
# labels = epochs.events[:,-1]-2
print(f"Data Size: {epochs_data.shape}")

# train data 
epochs_train = epochs.copy().crop(tmin= tmin, tmax=tmax)
epochs_train_data = epochs_train.get_data()
# epochs_train_data = np.transpose(epochs_train_data, axes = (1, 2, 0)) # Sklearn data = n_sam * n_freq
print(f"Train Size: {epochs_train_data.shape}")

Data Size: (88, 27, 961)
Train Size: (88, 27, 401)


In [44]:
M,N = epochs_train_data.shape[1:]
J,L = 3,8
scat_obj = Scattering2D(J=J,shape=(M,N),L=8,)

In [45]:
# Method1 [CH1,ch2,ch3]
# epochs_train_data = epochs_train_data.reshape((epochs_train_data.shape[0], -1))
# epochs_train_data.shape, labels.shape

# Method2 [T1, T2, T3 ]
epochs_train_data = np.transpose(epochs_train_data,axes=(0,2,1))
epochs_train_data = epochs_train_data.reshape((epochs_train_data.shape[0], -1))
epochs_train_data.shape, labels.shape


((88, 10827), (88,))

In [46]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [47]:
clf = SVC()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(epochs_train_data, labels)
pipe = Pipeline([('scatter', scat_obj), ('clf', clf)])
scores = cross_val_score(pipe, epochs_train_data, labels, cv = cv, verbose=False)

In [49]:
scores.mean()

0.5777777777777777

In [23]:
scat_obj.fit_transform()

['J',
 'L',
 'M',
 'M_padded',
 'N',
 'N_padded',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_n_features',
 '_doc_alias_call',
 '_doc_alias_name',
 '_doc_array',
 '_doc_array_n',
 '_doc_attr_out_type',
 '_doc_attrs_shape',
 '_doc_class',
 '_doc_frontend_paragraph',
 '_doc_has_out_type',
 '_doc_has_shape',
 '_doc_instantiation_shape',
 '_doc_param_out_type',
 '_doc_param_shape',
 '_doc_sample',
 '_doc_scattering',
 '_doc_shape',
 '_document',
 '_get_param_names',
 '_get_tags',
 '_instantiate_backend',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 'backend',
 'build

In [ ]:
from kymatio.sklearn import Scattering2D
from kymatio.sklearn import Scattering1D
from kymatio.scattering2d.filter_bank import filter_bank

In [ ]:
start_time = 0
end_time = 100
time = np.arange(start_time, end_time, 1/100)
theta = 0
frequency = 5
amplitude = 1
y = np.concatenate((np.sin(2 * np.pi * frequency * time[0:5000] + theta), np.sin(2 * np.pi * 10 * time[5000:] + theta)))
# figure(figsize=(20, 6), dpi=80)
plt.plot(time,y)

T =  y.shape[-1]
J,L = 4,16
scat_1D = Scattering1D(J,T,L)
Sx1 = scat_1D(y)

plt.figure()
for i in range(len(scat_1D.psi1_f)):
    plt.plot((np.abs(scat_1D.psi1_f[i][0])))
plt.figure()
for i in range(len(scat_1D.psi2_f)):
    plt.plot((np.abs(scat_1D.psi2_f[i][0])))
plt.figure()
for i in range(len(scat_1D.phi_f)):
    plt.plot((scat_1D.phi_f[0]))

meta  = scat_1D.meta()
order0 = np.where(meta['order'] == 0)
order1 = np.where(meta['order'] == 1)
order2 = np.where(meta['order'] == 2)

In [ ]:
M,N = epoch.get_data().shape[1:]
J,L = 3,8
scat2D = Scattering2D(J=J,shape=(M,N),L=L)
data = epoch['right'][0].get_data().squeeze(0)

In [ ]:
plt.figure()
for i in range(len(scat2D.psi[0])):
    plt.plot((scat2D.psi[i][0]))
plt.imshow((scat2D.psi[1][0]))

In [ ]:
filters = filter_bank(68,121,4,10)
filters